# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [21]:
# Import libraries
import pandas as pd

from pymongo import MongoClient

In [23]:
# Import data
df = pd.read_csv(f'ufo.csv')

ufo = MongoClient()

db = ufo.ufo

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ufo')

In [17]:
# Select data
df1 = df[(df.year>=2004) & (df['country']=='us')]

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47126 entries, 0 to 78508
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   47126 non-null  int64  
 1   datetime     47126 non-null  object 
 2   city         47126 non-null  object 
 3   state        47126 non-null  object 
 4   country      47126 non-null  object 
 5   shape        46560 non-null  object 
 6   duration     47126 non-null  float64
 7   total_time   47126 non-null  object 
 8   comments     47115 non-null  object 
 9   date_posted  47126 non-null  object 
 10  latitude     47126 non-null  float64
 11  longitude    47126 non-null  float64
 12  year         47126 non-null  int64  
 13  distance     47126 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 5.4+ MB


In [18]:
def data():    # funcion para realizar query a mongoDB
    query=db.companies.find({'$and':[{'$or': [{'category_code':'games_video'},
                                               {'category_code':'software'},
                                               {'category_code':'web'},
                                               {'category_code':'mobile'},
                                               {'category_code':'photo_video'},
                                               {'category_code':'ecommerce'},
                                               {'category_code':'search'},
                                               {'category_code':'network_hosting'}]},
                             {'founded_year': {'$gte':1999}},
                             {'offices': {'$ne':[]}}, 
                             {'offices.latitude': {'$ne':[]}},
                             {'offices.longitude': {'$ne':[]}},
                             {'offices.latitude': {'$ne':None}},
                             {'offices.longitude': {'$ne':None}},         
                             {'deadpooled_year':None},
                             {'number_of_employees': {'$ne':None}},
                             {'total_money_raised': {'$ne':None}}]},

                             {'name':1, '_id':0, 'offices.latitude':1, 'offices.longitude':1, 
                              'founded_year':1, 'category_code':1, 'number_of_employees':1,
                              'total_money_raised':1})


    df=pd.json_normalize(data=query, record_path='offices',
                      meta=['category_code', 'name', 'number_of_employees',
                            'founded_year', 'total_money_raised'])
    
    df.insert(2, 'geo_loc', pd.Series(list(range(len(df)))))
    df['geo_loc']=df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return df

In [ ]:
# Find the city